# Ejercicio 7
Considerando el pagelink_sample.csv, usando una representación de grafos realizar una función genérica que nos permita calcular la centralidad de un contenido cualquiera de la wikipedia mediante random walks. Mostrar el funcionamiento de la implementación con algún contenido incluido en el set de datos.

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [4]:
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Workaround problema pagelinks_sample.csv

In [5]:
pagelink_sample_pd = pd.read_csv('/content/drive/MyDrive/2021-2C/ORGANIZACION DE DATOS (7506)/TPs/TP2/Wikipedia_dataset-2c2021/pagelinks_sample.csv')

In [6]:
pagelink_sample_pd.head()

,pl_from,pl_namespace,pl_title,pl_from_namespace
0,4840492,10,Bandera_cÃ³digo_del_COI_TPE,2
1,950948,0,Esteban_DuÅ¡an,0
2,6418187,0,Insecta,0
3,3436623,0,Pleasure_to_Kill,0
4,4713066,0,DamiÃ¡n_Oliver,0


In [7]:
contents_pd = pd.read_csv('/content/drive/MyDrive/2021-2C/ORGANIZACION DE DATOS (7506)/TPs/TP2/Wikipedia_dataset-2c2021/contents.csv')

In [8]:
contents_pd.head()

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment
0,Wikipedia:Artículos solicitados,5,4,132533307,132516631.0,2021-01-19T18:40:34Z,Metrónomo,1708233.0,NaN,Así esta mejor
1,Andorra,7,0,138035057,138027475.0,2021-08-31T18:27:01Z,NaN,NaN,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido
2,Argentina,10,0,137966826,137825165.0,2021-08-28T17:17:54Z,Traitrich,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
3,Geografía de Andorra,15,0,137986788,133666383.0,2021-08-29T17:16:00Z,Lojwe,5338508.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
4,Demografía de Andorra,17,0,130869554,129413188.0,2020-11-12T19:56:15Z,Jialxv,3615366.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...


In [9]:
pagelink_reduce_pd = pagelink_sample_pd.loc[pagelink_sample_pd['pl_title'].isin(contents_pd['title'])]

In [10]:
pagelink_reduce_pd.shape

(20572932, 4)

In [11]:
pagelink_reduce_pd.to_parquet('pagelink_reduce.parquet')

In [12]:
del pagelink_sample_pd
del pagelink_reduce_pd
del contents_pd

Fin Workaround

In [13]:
df_pagelink = sqlContext.read.parquet('/content/pagelink_reduce.parquet', header=True, inferSchema=True)
df_pagelink

DataFrame[pl_from: bigint, pl_namespace: bigint, pl_title: string, pl_from_namespace: bigint, __index_level_0__: bigint]

In [14]:
rdd_pagelink = df_pagelink.rdd
rdd_pagelink.take(5)

[Row(pl_from=6418187, pl_namespace=0, pl_title='Insecta', pl_from_namespace=0, __index_level_0__=2),
 Row(pl_from=6571966, pl_namespace=0, pl_title='IMDb', pl_from_namespace=0, __index_level_0__=8),
 Row(pl_from=5461523, pl_namespace=4, pl_title='Vandalismo', pl_from_namespace=3, __index_level_0__=12),
 Row(pl_from=1098340, pl_namespace=0, pl_title='Wikidata', pl_from_namespace=0, __index_level_0__=13),
 Row(pl_from=6224910, pl_namespace=10, pl_title='Referencias', pl_from_namespace=3, __index_level_0__=16)]

In [15]:
df_contents = sqlContext.read.parquet('/content/drive/MyDrive/2021-2C/ORGANIZACION DE DATOS (7506)/TPs/TP2/Wikipedia_dataset-2c2021/contents.parquet', header=True, inferSchema=True)
df_contents

DataFrame[title: string, id: bigint, namespace: bigint, revision_id: bigint, parent_revision: double, revision_timestamp: string, revisor_username: string, revisor_id: double, revisor_ip: string, revisor_comment: string]

In [16]:
rdd_contents = df_contents.rdd
rdd_contents.take(5)

[Row(title='Wikipedia:Artículos solicitados', id=5, namespace=4, revision_id=132533307, parent_revision=132516631.0, revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id=1708233.0, revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id=7, namespace=0, revision_id=138035057, parent_revision=138027475.0, revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id=10, namespace=0, revision_id=137966826, parent_revision=137825165.0, revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id=4642283.0, revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id=15, namespace=0, revision_id=137986788, parent_revision=133666383.0, revision_timestamp='2021-08-29T17:16:00Z', revisor_username='Lojwe

In [17]:
contentsById = rdd_contents.map(lambda x: (x.title,x.id))
contentsById.take(5)

[('Wikipedia:Artículos solicitados', 5),
 ('Andorra', 7),
 ('Argentina', 10),
 ('Geografía de Andorra', 15),
 ('Demografía de Andorra', 17)]

In [18]:
pagelinkById = rdd_pagelink.map(lambda x: (x.pl_title,x.pl_from))
pagelinkById.take(5)

[('Insecta', 6418187),
 ('IMDb', 6571966),
 ('Vandalismo', 5461523),
 ('Wikidata', 1098340),
 ('Referencias', 6224910)]

In [19]:
grafo_rdd = pagelinkById.join(contentsById).map(lambda x: (x[1][0],x[1][1])).cache()
grafo_rdd.take(10)
# (from,to)

[(5851560, 1518135),
 (1742195, 1518135),
 (49876, 1518135),
 (9368, 1518135),
 (4893512, 1518135),
 (2806299, 1518135),
 (1078381, 1518135),
 (7010171, 1518135),
 (5006801, 1518135),
 (8559753, 8559807)]

In [20]:
grafo_rdd.count()

20573244

In [21]:
listaAdyacencias = grafo_rdd.groupByKey().map(lambda x: (x[0], list(x[1]))).cache()

In [22]:
listaAdyacencias.takeSample(False,1)[0]

(9093700, [682238])

In [23]:
listaAdyacencias.takeSample(False,1)[0]

(6427105, [736997, 30666, 750188, 874346, 633])

In [24]:
import random

Intento 1: con lista de adyacencias

In [25]:
def centralidad_aproximada(listaAdyacencias,walk_number=5,walk_length=100):
  cent = {}
  visitados = set()
  for i in range(walk_number):
    v,adyacentes = listaAdyacencias.takeSample(False,1)[0] # obtengo un vertice y 
                                                           # sus adyacentes aleatoriamente
                                                           # ejemplo: ('B', ['C', 'A', 'D']).
    if v not in visitados: 
        cent[v] = 1
        visitados.add(v)
    else: cent[v] += 1
    for j in range(walk_length-1):
      siguiente_nodo = listaAdyacencias.filter(lambda x: x[0] == v).collect()
      # puede ser que el nodo no apunte a nada y al filtrar me devuelva una lista vacía
      if not siguiente_nodo: 
        v,adyacentes = listaAdyacencias.takeSample(False,1)[0] # si me devuelve una lista vacía,
                                                               # obtengo otro nodo aleatoriamente
                                                               # y sus adyacentes.
      else:
        v,adyacentes = (siguiente_nodo[0][0],siguiente_nodo[0][1])
      w = random.choice(adyacentes)
      if w not in visitados:
        cent[w] = 1
        visitados.add(w)
      else: cent[w] += 1
      v = w

  return cent

In [26]:
cent = centralidad_aproximada(listaAdyacencias,1,20)

In [27]:
cent

{7720: 1,
 9777: 1,
 11565: 1,
 12303: 1,
 28813: 1,
 30666: 1,
 32724: 1,
 34245: 1,
 79244: 1,
 122314: 1,
 188209: 1,
 228736: 1,
 441613: 1,
 866572: 1,
 2400779: 1,
 4586329: 1,
 5291797: 2,
 5832445: 1,
 8900368: 1}

Intento 2: con lista de aristas

In [28]:
def centralidad_aproximada(grafoRDD,walk_number=5,walk_length=100):
  cent = {}
  visitados = set()
  for i in range(walk_number):
    v = grafoRDD.takeSample(False,1)[0][0]
    if v not in visitados:
        cent[v] = 1
        visitados.add(v)
    else: cent[v] += 1
    
    for j in range(walk_length-1):
      vecinos = grafoRDD.filter(lambda x: x[0] == v).map(lambda x: x[1]).collect()
      while not vecinos:
        v = grafoRDD.takeSample(False,1)[0][0]
        vecinos = grafoRDD.filter(lambda x: x[0] == v).map(lambda x: x[1]).collect()
      w = random.choice(vecinos)
    
      if w not in visitados:
        cent[w] = 1
        visitados.add(w)
      else: cent[w] += 1
      v = w
  return cent

In [29]:
cent = centralidad_aproximada(grafo_rdd,1,20)

In [30]:
cent

{10123: 1,
 14166: 1,
 21878: 1,
 22293: 1,
 23641: 1,
 30455: 2,
 30494: 1,
 30666: 1,
 34242: 1,
 34263: 1,
 42490: 1,
 42725: 1,
 42731: 1,
 42733: 2,
 62027: 1,
 92523: 1,
 327706: 1,
 377769: 1}

Intento 3: con lista de aristas 2

Esta alternativa tarda demasiado para formar el random walk. Sin embargo en teoría debería ocupar menos espacio en memoria, no se usa el .collect() hasta el final, cuando se quiere obtener la lista del random walks.

De ese camino que nos devuelve contaríamos las apariciones de cada vertice para averiguar la centralidad. 

In [31]:
def random_walk(grafoRDD, n = 10):
  '''
  recibe un rdd grafo con el formato (nodo1,nodo2) y un largo n
  devuelve un random walk de largo n. En caso de llegar a un nodo
  que no apunta a nada comienza de un nodo aleatorio nuevamente.
  '''
  camino = sc.parallelize(grafoRDD.takeSample(False,1)).map(lambda x: (x[1],x[0]))
  nodos_adyacentes = camino.leftOuterJoin(grafoRDD).map(lambda x: (x[0],x[1][1]))
  nodo_aleatorio = sc.parallelize(nodos_adyacentes.takeSample(False,1)).map(lambda x: (x[1],x[0]))
  camino = camino.union(nodo_aleatorio)
  for _ in range(n-2):
    nodos_adyacentes = nodo_aleatorio.leftOuterJoin(grafoRDD).map(lambda x: (x[0],x[1][1]))
    if nodos_adyacentes.collect()[0] == (None,None):
      nodo_aleatorio = sc.parallelize(grafoRDD.takeSample(False,1)).map(lambda x: (x[1],x[0]))
    else:
      nodo_aleatorio = sc.parallelize(nodos_adyacentes.takeSample(False,1)).map(lambda x: (x[1],x[0]))
    camino = camino.union(nodo_aleatorio)
  return camino.map(lambda x: x[1]).collect()

In [32]:
def centralidad(camino):
  cent = {}
  visitados = set()
  for v in camino:
    if v not in visitados:
      cent[v] = 1
      visitados.add(v)
    else: cent[v] += 1
  return cent